In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# installing the needed libraries

!pip install pyLDAvis --upgrade --quiet
!pip install opendatasets --upgrade --quiet
!pip install gensim --upgrade --quiet
!pip install nltk --upgrade --quiet
!pip install pyLDAvis --upgrade --quiet
!pip install spacy --upgrade --quiet
!pip install wordcloud --upgrade --quiet
!pip install textblob --upgrade --quiet
!pip install plotly  --upgrade --quiet
!pip install chart_studio --upgrade --quiet
!pip install cufflinks --upgrade --quiet
!pip install seaborn --upgrade --quiet
!pip install statsmodels --upgrade --quiet
!pip install bs4 --upgrade --quiet


In [5]:
# importing all relevant libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import re
from nltk.sentiment import SentimentIntensityAnalyzer
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from pprint import pprint
import spacy
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim


# natural language toolkit is used to download stopwords, vader lexicon & punkt
nltk.download(["stopwords","vader_lexicon","punkt"])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
# the relevant libraries are imported
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import re

customer_review = []

# each page is looped through to extract data to be appended to the customer_review list
for i in range(1,355):
  url = f'https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/'
  ba_reviews = requests.get(url)
  response = ba_reviews.content
  soup = BeautifulSoup(response, 'html.parser')

  first_article = soup.find('article', class_='comp comp_reviews-airline querylist position-content')
  review_articles = first_article.find_all('article', {'itemprop': 'review'})

  for review in review_articles:
      def reviewer_name():
        reviewer = review.find('span', itemprop="name").text
        if reviewer is None:
          return None
        else:
          return reviewer

      def general_rating():
        rating_element = review.find('span', itemprop="ratingValue")
        if rating_element is not None:
            return rating_element.text
        else:
            pass

      def comm_head():
        comment_header = review.find('h2',  class_="text_header").text
        if comment_header is None:
          return None
        else:
          return comment_header

      def comment_body():
        comment = review.find('div',  class_="text_content").text
        if comment is None:
          return None
        else:
          return comment

      def aircraft():
          first_td = review.find('td', class_='review-rating-header aircraft')
          if first_td is None:
              return None
          else:
              return first_td.find_next_sibling('td', class_="review-value").text

      def trav_type():
          traveller_td = review.find('td', class_='review-rating-header type_of_traveller')
          if traveller_td is None:
              return None
          else:
              return traveller_td.find_next_sibling('td', class_="review-value").text

      def seated_type():
          seat_td = review.find('td', class_='review-rating-header cabin_flown')
          if seat_td is None:
              return None
          else:
              return seat_td.find_next_sibling('td', class_="review-value").text

      def route_type():
          route_td = review.find('td', class_='review-rating-header route')
          if route_td is None:
              return None
          else:
              return route_td.find_next_sibling('td', class_="review-value").text

      def date_flown_func():
          date_flown_td = review.find('td', class_='review-rating-header date_flown')
          if date_flown_td is None:
              return None
          else:
              return date_flown_td.find_next_sibling('td', class_="review-value").text

      def seat_comfy():
          seat_comfort_header = review.find('td', class_="review-rating-header seat_comfort")
          if seat_comfort_header is None:
              return None
          else:
              seat_comfort = seat_comfort_header.parent
              filled_stars = seat_comfort.find_all('span', {'class': 'star fill'})
              return max([star.text for star in filled_stars])

      def wify():
          wifi_header = review.find('td', class_="review-rating-header wifi_and_connectivity")
          if wifi_header is None:
              return None
          else:
              wifi = wifi_header.parent
              filled_stars = wifi.find_all('span', {'class': 'star fill'})
              return max([star.text for star in filled_stars])

      def cabin_staff():
          cabin_staff_services = review.find('td', class_="review-rating-header cabin_staff_service")
          if cabin_staff_services is None:
              return None
          else:
              cabin_staff_service = cabin_staff_services.parent
              filled_stars = cabin_staff_service.find_all('span', {'class': 'star fill'})
              return max([star.text for star in filled_stars])

      def f_b():
          food_and_bev = review.find('td', class_="review-rating-header food_and_beverages")
          if food_and_bev is None:
              return None
          else:
              food_and_beverages = food_and_bev.parent
              filled_stars = food_and_beverages.find_all('span', class_='star fill')
              return max([star.text for star in filled_stars]) if filled_stars else None

      def recommended():
          first_td = review.find('td', class_="review-rating-header recommended")
          if first_td is None:
              return None
          else:
            second_td = first_td.find_next_sibling('td', class_="review-value rating-no")
            return 0 if second_td else 1

      def inflight_ent():
          inflight_enter = review.find('td', class_="review-rating-header inflight_entertainment")
          if inflight_enter is None:
              return None
          else:
              flight_ent = inflight_enter.parent
              filled_stars = flight_ent.find_all('span', class_='star fill')
              return max([star.text for star in filled_stars]) if filled_stars else None

      def ground_service():
          ground_services = review.find('td', class_="review-rating-header ground_service")
          if ground_services is None:
              return None
          else:
              g_service = ground_services.parent
              filled_stars = g_service.find_all('span', {'class': 'star fill'})
              return max([star.text for star in filled_stars])

      def val_money():
          value_for_money = review.find('td', class_="review-rating-header value_for_money")
          v_for_money = value_for_money.parent
          filled_stars = v_for_money.find_all('span', class_ = "star fill")
          return max([star.text for star in filled_stars]) if filled_stars else None

      customer_review.append([reviewer_name(), general_rating(), comm_head(), comment_body(),
                              aircraft(), trav_type(), seated_type(), route_type(), date_flown_func(),
                              seat_comfy(), wify(), cabin_staff(), f_b(), inflight_ent(), ground_service(), val_money(), recommended()])




In [9]:
# customer_view list is used to create a dataframe
df = pd.DataFrame(customer_review, columns=['reviewer', 'overall rating', 'comment head', 'comment body',
                                            'aircraft', 'travel type', 'seat type', 'route', 'date flown',
                                            'seat comfort', 'wifi', 'cabin services', 'food & drink', 'entertainment',
                                            'ground services', 'value_money', 'recommended'])
df.head()


,reviewer,overall rating,comment head,comment body,aircraft,travel type,seat type,route,date flown,seat comfort,wifi,cabin services,food & drink,entertainment,ground services,value_money,recommended
0,J Tydeman,4,"""Very second rate shoddy airline""",Not Verified | Flew with BA to Punta Cana. To...,None,Couple Leisure,Economy Class,London Gatwick to Punta Cana,October 2023,3,None,3,3,2,2,2,0
1,Hugh Kilpatrick,1,"""some sort of medieval torture""",✅ Trip Verified | They downgraded me from bu...,None,Solo Leisure,Business Class,London to Hong Kong,July 2023,1,None,5,3,3,5,1,0
2,Glenn Taylor-Biffen,1,"""Never again BA""",✅ Trip Verified | The already delayed aircra...,A320,Couple Leisure,Economy Class,Catania to Gatwick,October 2023,1,None,1,1,None,1,1,0
3,Glenn Taylor-Biffen,2,"""BA Gatwick are not BA""",✅ Trip Verified | Caught up in the Gatwick c...,None,Couple Leisure,Economy Class,Gatwick to Venice,September 2023,None,None,None,None,None,1,1,0
4,S Harten,1,"""Boarding flight took an hour""",✅ Trip Verified | BA has a real problem with...,A321,Solo Leisure,Economy Class,Berlin to London,October 2023,3,None,1,1,None,1,1,0


In [11]:
# regex is used to drop some unwanted text/characters from the comment column
pattern = r'✅ Trip Verified \| |Not Verified \| |❎ Not Verified \| |✅ Verified Review \| |\✅ Trip Verified \|'
df['comment body'] = df['comment body'].str.replace(pattern, '', regex=True)

df.head()

,reviewer,overall rating,comment head,comment body,aircraft,travel type,seat type,route,date flown,seat comfort,wifi,cabin services,food & drink,entertainment,ground services,value_money,recommended
0,J Tydeman,4,"""Very second rate shoddy airline""",Flew with BA to Punta Cana. Took the trouble ...,None,Couple Leisure,Economy Class,London Gatwick to Punta Cana,October 2023,3,None,3,3,2,2,2,0
1,Hugh Kilpatrick,1,"""some sort of medieval torture""",They downgraded me from business to premium ...,None,Solo Leisure,Business Class,London to Hong Kong,July 2023,1,None,5,3,3,5,1,0
2,Glenn Taylor-Biffen,1,"""Never again BA""",The already delayed aircraft landed and apar...,A320,Couple Leisure,Economy Class,Catania to Gatwick,October 2023,1,None,1,1,None,1,1,0
3,Glenn Taylor-Biffen,2,"""BA Gatwick are not BA""",Caught up in the Gatwick cancellation fiasc...,None,Couple Leisure,Economy Class,Gatwick to Venice,September 2023,None,None,None,None,None,1,1,0
4,S Harten,1,"""Boarding flight took an hour""",BA has a real problem with boarding it's fl...,A321,Solo Leisure,Economy Class,Berlin to London,October 2023,3,None,1,1,None,1,1,0


In [12]:
df.columns

Index(['reviewer', 'overall rating', 'comment head', 'comment body',
       'aircraft', 'travel type', 'seat type', 'route', 'date flown',
       'seat comfort', 'wifi', 'cabin services', 'food & drink',
       'entertainment', 'ground services', 'value_money', 'recommended'],
      dtype='object')